In [1]:
import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  # for manipulating text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for interchange across GIS programs
import rtree  # supports geospatial join
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
import descartes
import sys
import sklearn
from shapely.ops import nearest_points
from datetime import datetime as dt, date
sys.path.append('/Users/saraprice/Documents/NYU/Fall_2020/DS_GA_1001/final_project/wildfires-1001/code/functions/')
from gis_processing import *

In [3]:
data_dir = '/Users/saraprice/Documents/NYU/Fall_2020/DS_GA_1001/wildfires-1001/data'
target_data = gpd.GeoDataFrame()
for i in np.arange(1,4):
    t_df = pd.read_pickle(os.path.join(data_dir, f'clean_data/target_full_{i}.pkl'))
    target_data = target_data.append(t_df)

In [7]:
full_fire_data = gpd.read_file(os.path.join(data_dir, 'clean_data/fire_gis/fire_gis.dbf'))
full_fire_data.head()

,AGENCY,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,Shape_Leng,Shape_Area,FIRE_KEY,YEAR,geometry
0,CCO,OCTOBER,00246393,2007-10-21,2007-10-23,14.0,NaN,25.736713,8.0,1.0,00233414,1902.439051,1.041528e+05,OCTOBER_2007_00246393_00233414,2007,"POLYGON ((-13191194.231 4080267.936, -13191133..."
1,CCO,MAGIC,00233077,2007-10-22,2007-10-25,14.0,NaN,2824.877197,8.0,1.0,00233077,20407.965662,1.143187e+07,MAGIC_2007_00233077_00233077,2007,"POLYGON ((-13200763.900 4085308.071, -13200737..."
2,USF,RANCH,00000166,2007-10-20,2007-11-15,2.0,54716.0,58410.335938,7.0,1.0,00000166,169150.715690,2.363782e+08,RANCH_2007_00000166_00000166,2007,"POLYGON ((-13219907.361 4109183.199, -13219694..."
3,CCO,EMMA,00201384,2007-09-11,2007-09-11,14.0,NaN,172.214951,8.0,1.0,00201384,6117.777086,6.969292e+05,EMMA_2007_00201384_00201384,2007,"POLYGON ((-13143800.871 4096406.674, -13143793..."
4,CCO,CORRAL,00259483,2007-11-24,2007-11-27,14.0,NaN,4707.997070,8.0,1.0,00259483,22907.182174,1.905259e+07,CORRAL_2007_00259483_00259483,2007,"POLYGON ((-13218530.800 4039723.471, -13218504..."


In [6]:
## Historical Fire Features
### Fire in the last month, year, 5 years, 10 years

,date,month_id,month_start,month_end,week_id,week_start,week_end,GRID_ID,FIRE_AREA,FIRE_KEY,...,start_date,end_date,geometry,FIRE_GRID_INT_AREA,FIRE_ACRES,Y_fire_class_size,NAME,COUNTYFP,COUNTY_ARE,GRID_AREA
0,1990-01-01,1990_1,1990-01-01,1990-01-31,1990_1,1990-01-01,1990-01-07,0.0,NaN,NaN,...,NaT,NaT,None,NaN,0.0,0,Del Norte,015,5.725577e+09,2.665977e+09
1,1990-01-02,1990_1,1990-01-01,1990-01-31,1990_1,1990-01-01,1990-01-07,0.0,NaN,NaN,...,NaT,NaT,None,NaN,0.0,0,Del Norte,015,5.725577e+09,2.665977e+09
2,1990-01-03,1990_1,1990-01-01,1990-01-31,1990_1,1990-01-01,1990-01-07,0.0,NaN,NaN,...,NaT,NaT,None,NaN,0.0,0,Del Norte,015,5.725577e+09,2.665977e+09
3,1990-01-04,1990_1,1990-01-01,1990-01-31,1990_1,1990-01-01,1990-01-07,0.0,NaN,NaN,...,NaT,NaT,None,NaN,0.0,0,Del Norte,015,5.725577e+09,2.665977e+09
4,1990-01-05,1990_1,1990-01-01,1990-01-31,1990_1,1990-01-01,1990-01-07,0.0,NaN,NaN,...,NaT,NaT,None,NaN,0.0,0,Del Norte,015,5.725577e+09,2.665977e+09
